In [64]:
import quandl
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import warnings
import pickle
import sklearn.cluster as cluster
import itertools
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [11]:
pip install fundamentalanalysis==0.2.13

  Attempting uninstall: fundamentalanalysis
    Found existing installation: fundamentalanalysis 0.3.1
    Uninstalling fundamentalanalysis-0.3.1:
      Successfully uninstalled fundamentalanalysis-0.3.1
Note: you may need to restart the kernel to use updated packages.


In [12]:
import FundamentalAnalysis as fa

In [ ]:
api_key = ''

In [17]:
# Show the available companies
companies = fa.available_companies(api_key)
companies

,name,price,exchange,exchangeShortName,type
symbol,,,,,
PMGOLD.AX,Perth Mint Gold,17.9400,Australian Securities Exchange,ASX,etf
3692.HK,Hansoh Pharmaceutical Group Company Limited,17.1600,HKSE,HKSE,stock
1815.HK,CSMall Group Limited,0.1990,HKSE,HKSE,stock
0187.HK,Beijing Jingcheng Machinery Electric Company L...,3.3800,HKSE,HKSE,stock
0175.HK,Geely Automobile Holdings Limited,14.5800,HKSE,HKSE,stock
...,...,...,...,...,...
MDNWF,Mednow Inc.,0.0200,Other OTC,PNK,stock
STA.L,Star Phoenix Group Ltd,1.0500,London Stock Exchange,LSE,stock
HALO.NE,Halo Collective Inc.,0.0200,NEO,NEO,stock


In [151]:
companies[(companies['exchangeShortName']=='NYSE')]


,name,price,exchange,exchangeShortName,type
symbol,,,,,
CTVA,"Corteva, Inc.",61.7400,New York Stock Exchange,NYSE,stock
MBC,"MasterBrand, Inc.",17.6700,New York Stock Exchange,NYSE,stock
SMR,NuScale Power Corporation,21.4100,New York Stock Exchange,NYSE,stock
GMED,"Globus Medical, Inc.",82.7100,New York Stock Exchange,NYSE,stock
ICL,ICL Group Ltd,4.2700,New York Stock Exchange,NYSE,stock
...,...,...,...,...,...
CSTA,Constellation Acquisition Corp I,11.0950,New York Stock Exchange,NYSE,stock
MBAC-WT,M3-Brigade Acquisition II Corp.,0.0026,New York Stock Exchange,NYSE,stock
GOGN-WT,GoGreen Investments Corporation,1.0500,New York Stock Exchange,NYSE,stock


In [150]:
companies[(companies['exchangeShortName']=='NYSE')]['exchange'].unique()

array(['New York Stock Exchange', 'American Stock Exchange'], dtype=object)

In [29]:
exchange

array(['ASX', 'HKSE', 'JKT', 'SES', 'CPH', 'SHZ', 'SHH', 'JPX', 'KLS',
       None, 'VSE', 'KSC', 'TAI', 'KOE', 'TWO', 'NZE', 'EURONEXT',
       'NASDAQ', 'PNK', 'AMEX', 'XETRA', 'NEO', 'SAO', 'TSX', 'NYSE',
       'BVC', 'CBOE', 'TSXV', 'OTC', 'CNQ', 'MEX', 'STU', 'OQX', 'BER',
       'MCX', 'HAM', 'BUE', 'LSE', 'SET', 'IOB', 'SGO', 'DUS', 'BME',
       'VIE', 'MIL', 'AMS', 'BRU', 'SIX', 'STO', 'HEL', 'DXE', 'MUN',
       'JNB', 'BUD', 'WSE', 'TLV', 'OSL', 'ICE', 'PRA', 'ATH', 'IST',
       'AQS', 'RIS', 'CAI', 'SAU', 'DFM', 'BSE', 'NSE', 'KUW', 'DOH',
       'OEM', 'EGX'], dtype=object)

In [157]:
filtered_df = companies[(companies['exchangeShortName'].isin(['NASDAQ','NYSE'])) & (companies['type'] == 'stock')& companies['exchange'].isin(['NASDAQ Global Select Market','NASDAQ Global Market','New York Stock Exchange'])]

In [159]:
filtered_df

,name,price,exchange,exchangeShortName,type
symbol,,,,,
CTVA,"Corteva, Inc.",61.7400,New York Stock Exchange,NYSE,stock
MBC,"MasterBrand, Inc.",17.6700,New York Stock Exchange,NYSE,stock
SMR,NuScale Power Corporation,21.4100,New York Stock Exchange,NYSE,stock
GMED,"Globus Medical, Inc.",82.7100,New York Stock Exchange,NYSE,stock
ICL,ICL Group Ltd,4.2700,New York Stock Exchange,NYSE,stock
...,...,...,...,...,...
MBAC-WT,M3-Brigade Acquisition II Corp.,0.0026,New York Stock Exchange,NYSE,stock
GOGN-WT,GoGreen Investments Corporation,1.0500,New York Stock Exchange,NYSE,stock
UNVR,Univar Solutions Inc.,36.1400,New York Stock Exchange,NYSE,stock


In [160]:
filtered_df = filtered_df.reset_index()

In [185]:
symbols = filtered_df['symbol'].unique().tolist()

In [186]:
symbols

['CTVA',
 'MBC',
 'SMR',
 'GMED',
 'ICL',
 'MS',
 'ABT',
 'WOLF',
 'DD',
 'MRK',
 'HPE',
 'AGM-A',
 'RCL',
 'HVT-A',
 'IR',
 'PG',
 'CVX',
 'XOM',
 'BRK-B',
 'T',
 'LOW',
 'DIS',
 'W',
 'UPS',
 'ZKH',
 'SG',
 'AM',
 'CMG',
 'CAVA',
 'RDDT',
 'MX',
 'AR',
 'SKT',
 'ARCH',
 'BHR-PB',
 'DXYZ',
 'NOVA',
 'CON',
 'APLE',
 'PACS',
 'GOLD',
 'WTM',
 'BRK-A',
 'F',
 'CPRI',
 'AMTM',
 'WFC',
 'EBR-B',
 'FICO',
 'NVR',
 'CCM',
 'FCX',
 'KLC',
 'EMKR',
 'EVRI',
 'FBRT',
 'FSLY',
 'SHG',
 'TSLX',
 'GBTG',
 'DECK',
 'EGO',
 'DNMR',
 'EVTL',
 'LAW',
 'MDU',
 'OOMA',
 'DQ',
 'ATEN',
 'YETI',
 'PAC',
 'RSI',
 'STNG',
 'EVH',
 'NOV',
 'NR',
 'NSA',
 'NEE-PR',
 'FC',
 'ENR',
 'EOLS',
 'PATH',
 'NUS',
 'IEX',
 'OLO',
 'DOLE',
 'NVST',
 'GSBD',
 'SKM',
 'PB',
 'PFE',
 'PFS',
 'EIX',
 'PRGO',
 'INOD',
 'IRM',
 'LAC',
 'SN',
 'SAVE',
 'FPI',
 'PKE',
 'UAN',
 'BFAM',
 'FF',
 'MYN',
 'AX',
 'AVAL',
 'BBVA',
 'LAAC',
 'LUV',
 'AXS-PE',
 'BOWL',
 'BBY',
 'BCO',
 'KNX',
 'KO',
 'AMPY',
 'AMX',
 'APLT',
 'BE',
 '

In [187]:
symbols = [s for s in symbols if '-' not in s ]

In [189]:
len(symbols)

4492

In [51]:
# get the balance_sheet_per_year
balance_sheet_annually = fa.balance_sheet_statement('AAPL', api_key, period="annual")
balance_sheet_annually 

,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
cik,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,...,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193
fillingDate,2024-11-01,2023-11-03,2022-10-28,2021-10-29,2020-10-30,2019-10-31,2018-11-05,2017-11-03,2016-10-26,2015-10-28,...,1994-12-13,1993-09-30,1992-09-30,1991-09-30,1990-09-30,1989-09-30,1988-09-30,1987-09-30,1986-09-30,1985-09-30
acceptedDate,2024-11-01 06:01:36,2023-11-02 18:08:27,2022-10-27 18:01:14,2021-10-28 18:04:28,2020-10-29 18:06:25,2019-10-30 18:12:36,2018-11-05 08:01:40,2017-11-03 08:01:37,2016-10-26 16:42:16,2015-10-28 16:31:09,...,1994-12-13 00:00:00,1993-09-30 00:00:00,1992-09-30 00:00:00,1991-09-30 00:00:00,1990-09-30 00:00:00,1989-09-30 00:00:00,1988-09-30 00:00:00,1987-09-30 00:00:00,1986-09-30 00:00:00,1985-09-30 00:00:00
calendarYear,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,...,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
cashAndCashEquivalents,29943000000,29965000000,23646000000,34940000000,38016000000,48844000000,25913000000,20289000000,20484000000,21120000000,...,1203000000,676413000,498600000,604100000,374700000,438300000,545700000,565100000,576200000,337000000
shortTermInvestments,35228000000,31590000000,24658000000,27699000000,52927000000,51713000000,40388000000,53892000000,46671000000,20481000000,...,55000000,215890000,0,0,0,0,0,0,0,0
cashAndShortTermInvestments,65171000000,61555000000,48304000000,62639000000,90943000000,100557000000,66301000000,74181000000,67155000000,41601000000,...,1258000000,892303000,498600000,604100000,374700000,438300000,545700000,565100000,576200000,337000000
netReceivables,66243000000,60985000000,60932000000,51506000000,37445000000,45804000000,48995000000,35673000000,29299000000,30343000000,...,1581000000,1381946000,1087200000,907200000,761900000,792800000,638800000,405600000,263100000,220200000


In [52]:
cash_flow_statement_annually = fa.cash_flow_statement('AAPL', api_key, period="annual")
cash_flow_statement_annually

,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,1989
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
cik,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,...,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193,0000320193
fillingDate,2024-11-01,2023-11-03,2022-10-28,2021-10-29,2020-10-30,2019-10-31,2018-11-05,2017-11-03,2016-10-26,2015-10-28,...,1998-12-23,1997-12-05,1996-12-19,1995-12-19,1994-12-13,1993-09-30,1992-09-30,1991-09-30,1990-09-30,1989-09-30
acceptedDate,2024-11-01 06:01:36,2023-11-02 18:08:27,2022-10-27 18:01:14,2021-10-28 18:04:28,2020-10-29 18:06:25,2019-10-30 18:12:36,2018-11-05 08:01:40,2017-11-03 08:01:37,2016-10-26 16:42:16,2015-10-28 16:31:09,...,1998-12-23 00:00:00,1997-12-05 00:00:00,1996-12-19 00:00:00,1995-12-19 00:00:00,1994-12-13 00:00:00,1993-09-30 00:00:00,1992-09-30 00:00:00,1991-09-30 00:00:00,1990-09-30 00:00:00,1989-09-30 00:00:00
calendarYear,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,1989
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,...,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
netIncome,93736000000,96995000000,99803000000,94680000000,57411000000,55256000000,59531000000,48351000000,45687000000,53394000000,...,309000000,-1045000000,-816000000,424000000,310000000,87000000,530373000,309800000,474900000,454000000
depreciationAndAmortization,11445000000,11519000000,11104000000,11284000000,11056000000,12547000000,10903000000,10157000000,10505000000,11257000000,...,111000000,118000000,116000000,127000000,168000000,166000000,217182000,204400000,202700000,124800000
deferredIncomeTax,0,0,895000000,-4774000000,-215000000,-340000000,-32590000000,5966000000,4938000000,1382000000,...,1000000,-50000000,-437000000,73000000,40800000,19000000,100900000,8000000,138900000,111300000
stockBasedCompensation,11688000000,10833000000,9038000000,7906000000,6829000000,6068000000,5340000000,4840000000,4210000000,3586000000,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# Show Key Metrics
key_metrics_annually = fa.key_metrics('AAPL', api_key, period="annual")
key_metrics_annually

,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
calendarYear,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,...,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenuePerShare,25.363711,24.344473,24.317273,21.903541,15.820258,14.085283,13.399334,10.984444,9.854053,10.15548,...,0.690971,0.597886,0.516551,0.469102,0.395022,0.367868,0.279699,0.181339,0.132321,0.139932
netIncomePerShare,6.08,6.160669,6.154614,5.669029,3.308587,2.991446,3.003354,2.316885,2.087758,2.320098,...,0.023311,0.006521,0.03866,0.023036,0.03375,0.031607,0.0275,0.014821,0.010714,0.004464
operatingCashFlowPerShare,7.670312,7.021175,7.532763,6.229346,4.64923,3.756685,3.906565,3.077536,3.026557,3.531203,...,0.055421,-0.048793,0.067132,0.00957,0.068502,0.035318,None,None,None,None
freeCashFlowPerShare,7.057551,6.32511,6.872426,5.565624,4.228014,3.188508,3.23492,2.464425,2.407454,3.032022,...,0.04342,-0.064765,0.052925,-0.006662,0.052562,0.018679,None,None,None,None
cashPerShare,4.227188,3.909686,2.978793,3.750553,5.241031,5.443948,3.344902,3.554608,3.068781,1.807664,...,0.094598,0.066879,0.036344,0.044919,0.026629,0.030514,0.037489,0.038508,0.040088,0.024583
bookValuePerShare,3.693949,3.947224,3.124822,3.777557,3.765477,4.898834,5.405593,6.423269,5.860593,5.186262,...,0.179196,0.15188,0.159445,0.131366,0.102821,0.103433,0.068932,0.057003,0.048291,0.040157
tangibleBookValuePerShare,4.958713,3.947224,3.124822,3.777557,3.765477,4.898834,5.405593,6.039206,5.466685,4.794799,...,0.179196,0.15188,0.159445,0.131366,0.102821,0.103433,0.068932,0.057003,0.048291,0.040157
shareholdersEquityPerShare,3.693949,3.947224,3.124822,3.777557,3.765477,4.898834,5.405593,6.423269,5.860593,5.186262,...,0.179196,0.15188,0.159445,0.131366,0.102821,0.103433,0.068932,0.057003,0.048291,0.040157


In [54]:
# Show a large set of in-depth ratios
financial_ratios_annually = fa.financial_ratios('AAPL', api_key, period="annual")
financial_ratios_annually

,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
calendarYear,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,...,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
currentRatio,0.867313,0.988012,0.879356,1.074553,1.363604,1.540126,1.132926,1.276063,1.352669,1.108771,...,2.302469,1.729748,2.496247,2.352806,2.340117,2.562716,2.155725,2.731147,3.168645,2.783006
quickRatio,0.826007,0.944442,0.847235,1.022115,1.325072,1.501286,1.098802,1.227905,1.325684,1.07963,...,1.742798,1.129035,2.089302,1.80092,1.993963,2.031721,1.597751,2.259453,2.837747,2.217671
cashRatio,0.169753,0.206217,0.153563,0.278449,0.36071,0.462022,0.223525,0.201252,0.259271,0.262002,...,0.618827,0.269693,0.349772,0.496344,0.364849,0.489557,0.659775,1.180489,1.754033,1.140826
daysOfSalesOutstanding,61.83256,58.075649,56.400205,51.390969,49.787534,64.258765,67.332499,56.800671,49.592768,47.387609,...,62.801265,63.233081,55.997742,52.486685,50.031214,54.763815,57.268262,55.632633,50.492402,41.898035
daysOfInventoryOutstanding,12.642571,10.791292,8.075698,11.276593,8.741883,9.263639,8.817631,12.563631,5.923304,6.120288,...,58.016791,108.195028,56.101028,78.840563,53.986894,67.517899,88.044899,67.24076,47.232738,56.649628
operatingCycle,74.47513,68.866941,64.475903,62.667561,58.529418,73.522404,76.15013,69.364302,55.516073,53.507897,...,120.818056,171.42811,112.09877,131.327249,104.018108,122.281714,145.31316,122.873394,97.72514,98.547663
daysOfPayablesOutstanding,119.658477,106.721468,104.685277,93.851071,91.04819,104.314077,124.570214,114.488188,103.613369,92.468716,...,47.031994,53.329339,41.285173,41.914493,51.724152,47.464202,0,0,0,0
cashConversionCycle,-45.183347,-37.854527,-40.209374,-31.18351,-32.518772,-30.791673,-48.420084,-45.123887,-48.097297,-38.96082,...,73.786062,118.098771,70.813597,89.412755,52.293956,74.817512,145.31316,122.873394,97.72514,98.547663


In [59]:
metrics_fc = financial_ratios_annually.index.to_list()

In [60]:
metrics_fc

['calendarYear',
 'period',
 'currentRatio',
 'quickRatio',
 'cashRatio',
 'daysOfSalesOutstanding',
 'daysOfInventoryOutstanding',
 'operatingCycle',
 'daysOfPayablesOutstanding',
 'cashConversionCycle',
 'grossProfitMargin',
 'operatingProfitMargin',
 'pretaxProfitMargin',
 'netProfitMargin',
 'effectiveTaxRate',
 'returnOnAssets',
 'returnOnEquity',
 'returnOnCapitalEmployed',
 'netIncomePerEBT',
 'ebtPerEbit',
 'ebitPerRevenue',
 'debtRatio',
 'debtEquityRatio',
 'longTermDebtToCapitalization',
 'totalDebtToCapitalization',
 'interestCoverage',
 'cashFlowToDebtRatio',
 'companyEquityMultiplier',
 'receivablesTurnover',
 'payablesTurnover',
 'inventoryTurnover',
 'fixedAssetTurnover',
 'assetTurnover',
 'operatingCashFlowPerShare',
 'freeCashFlowPerShare',
 'cashPerShare',
 'payoutRatio',
 'operatingCashFlowSalesRatio',
 'freeCashFlowOperatingCashFlowRatio',
 'cashFlowCoverageRatios',
 'shortTermCoverageRatios',
 'capitalExpenditureCoverageRatio',
 'dividendPaidAndCapexCoverageRatio

In [61]:
selected_columns = [
    'currentRatio', 'quickRatio',                      # Liquidity Ratios
    'inventoryTurnover', 'fixedAssetTurnover',         # Operating Efficiency Ratios
    'grossProfitMargin', 'netProfitMargin',            # Profitability Ratios
    'returnOnAssets', 'returnOnEquity',                # Profitability Ratios
    'debtEquityRatio', 'interestCoverage'              # Leverage Ratios
]

In [62]:
filtered_financial_ratios_annually = financial_ratios_annually.loc[selected_columns]

In [63]:
filtered_financial_ratios_annually 

,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985
currentRatio,0.867313,0.988012,0.879356,1.074553,1.363604,1.540126,1.132926,1.276063,1.352669,1.108771,...,2.302469,1.729748,2.496247,2.352806,2.340117,2.562716,2.155725,2.731147,3.168645,2.783006
quickRatio,0.826007,0.944442,0.847235,1.022115,1.325072,1.501286,1.098802,1.227905,1.325684,1.07963,...,1.742798,1.129035,2.089302,1.80092,1.993963,2.031721,1.597751,2.259453,2.837747,2.217671
inventoryTurnover,28.870711,33.823567,45.197331,32.367933,41.753016,39.401364,41.394338,29.052111,61.621013,59.637718,...,6.291282,3.373538,6.50612,4.629597,6.7609,5.405974,4.145612,5.428255,7.727691,6.443114
fixedAssetTurnover,8.560311,7.04879,7.506148,7.386214,6.055122,6.960619,6.430249,6.785484,7.983673,10.400739,...,13.776234,12.094683,15.332107,14.082143,13.958815,15.810892,19.630665,20.407209,17.72507,21.220133
grossProfitMargin,0.462063,0.441311,0.433096,0.417794,0.382332,0.378178,0.383437,0.384699,0.39076,0.400599,...,0.255076,0.362831,0.46741,0.507085,0.567591,0.513626,0.530088,0.539401,0.558336,0.439087
netProfitMargin,0.239713,0.253062,0.253096,0.258818,0.209136,0.212381,0.224142,0.210924,0.211868,0.228458,...,0.033737,0.010906,0.074843,0.049106,0.085438,0.08592,0.09832,0.081733,0.080972,0.031903
returnOnAssets,0.256825,0.275098,0.282924,0.269742,0.177256,0.16323,0.162775,0.128826,0.142024,0.183898,...,0.058457,0.016823,0.125571,0.088676,0.159593,0.165458,0.192258,0.147168,0.132747,0.065371
returnOnEquity,1.645935,1.56076,1.969589,1.500713,0.878664,0.610645,0.555601,0.360702,0.356237,0.447355,...,0.130088,0.042934,0.242467,0.175355,0.328242,0.30558,0.398944,0.260012,0.22187,0.111172
debtEquityRatio,1.872327,1.787533,2.614462,2.163925,1.87144,1.194048,1.068467,0.862981,0.678617,0.538964,...,0.250525,0.409745,0.084347,0.084112,0.084739,0.038231,0.127467,0,0,0
interestCoverage,0,29.062039,40.749574,41.190548,23.072746,17.877517,21.882099,26.407232,42.971841,97.175989,...,13.171109,-1.383959,-8.105634,-4.26813,-5.353383,-2.883182,-8.651325,-4.775064,-3.772414,2.702752


In [85]:
# get fundamental data from api for each symbol

In [223]:
def get_fc_data(year, fc_factors, ticker):
    # get the financial ratio
    fc_data = fa.financial_ratios(ticker, api_key, period="annual")
    
    # check if the selected_columns all included in fc data  
    available_factors = [factor for factor in fc_factors if factor in fc_data.index]
    
    if not available_factors:
        print(f"No available data for {ticker} in {year}.")
        return pd.DataFrame()
    
    
    # check if the given company have data of giving year
    year_str = str(year)
    if year_str not in fc_data.columns:
        print(f"Year {year} data is not available for {ticker}.")
        return pd.DataFrame()
    
    # start to extract fc data
    fc = fc_data.loc[available_factors, [year_str]].rename(columns={year_str: ticker})
    
    # delete if null or zero
    if fc[ticker].isnull().sum()  >= 1:
        print(f"NullValue{ticker} in {year}.")
        return pd.DataFrame()
#     if (fc[ticker].abs() < 1e-10).sum() >= 1:
#         print(f"ZeroValue{ticker} in {year}.")
#         return pd.DataFrame() 
    return fc

In [219]:
get_fc_data(2023,selected_columns,'KVYO')

,KVYO
currentRatio,6.035179
quickRatio,6.035179
inventoryTurnover,0
fixedAssetTurnover,8.678829
grossProfitMargin,0.745182
netProfitMargin,-0.441532
returnOnAssets,-0.283029
returnOnEquity,-0.336949
debtEquityRatio,0.056384
interestCoverage,0


In [117]:
# get fundamental data for all of the symbols

In [259]:
def download_fc_all(year):
    tickers = symbols
    year = year
    fc_factors = selected_columns
    fc = get_fc_data(year,fc_factors,tickers[3500])
    for ticker in tqdm(tickers[3501:3700]):
        ticker_fc = get_fc_data(year,fc_factors,ticker)
        if ticker_fc.empty:
            continue
        fc = fc.join(ticker_fc)   
    return fc

In [260]:
fc_data_2023_4 = download_fc_all(2023)

NullValueNCR in 2023.


  0%|          | 0/199 [00:00<?, ?it/s]

Year 2023 data is not available for ABTX.
No available data for IGZ in 2023.
Year 2023 data is not available for SCMA.
Year 2023 data is not available for MBT.
Year 2023 data is not available for VYGG.
Year 2023 data is not available for PCXCU.
Year 2023 data is not available for TMAC.
Year 2023 data is not available for SHLX.
Year 2023 data is not available for GIACW.
Year 2023 data is not available for ESM.
Year 2023 data is not available for OG.
Year 2023 data is not available for FMTX.
Year 2023 data is not available for DAOOU.
Year 2023 data is not available for LITT.
Year 2023 data is not available for MOHO.
No available data for PEI in 2023.
No available data for BOAC in 2023.
No available data for AJAX in 2023.
Year 2023 data is not available for AONE.
Year 2023 data is not available for ARD.
Year 2023 data is not available for RACY.
Year 2023 data is not available for FLYA.
Year 2023 data is not available for HIL.
Year 2023 data is not available for ARGU.
Year 2023 data is not

In [261]:
cols_with_zero = ( fc_data_2023_4 < 1e-10).any()

# 保留不包含 0 的列
fc_data_2023_4 = fc_data_2023_4.loc[:, ~cols_with_zero]

fc_data_2023_4

,NVTSW,DCUE,DO,AUD,CRY,ECCB,ACAX,ACAXU,RENEU,LPI,...,BAMR,GPS,BLL,MAAQR,LAXXU,LAXXW,LSPRW,LSPR,BACA,ELAT


In [238]:
fc_data_2023 

,CTVA,MBC,SMR,GMED,ICL,MS,ABT,WOLF,DD,MRK,...,CAMT,SMMT,EBR,BGS,SIGA,VZ,CRH,SDIG,EVGN,EXPI
currentRatio,1.56211,1.93818,1.767088,4.881837,1.529189,0.411079,1.637887,5.477716,2.425436,1.251965,...,5.655988,9.297329,1.763531,3.059333,4.416133,0.691693,1.686308,0.504152,4.963854,1.881354
quickRatio,0.899318,1.22324,1.767088,2.720141,0.883624,3.937052,1.163211,4.982928,1.732408,1.004515,...,4.767383,9.177702,1.747821,0.891113,3.229492,0.653045,1.257765,0.384299,4.720046,1.881354
inventoryTurnover,1.43789,7.407126,18961000,0.646329,3.004617,0,2.735921,1.961527,3.928738,2.536332,...,1.952645,0.83982,44.311406,2.867482,0.27757,26.683034,5.356793,23.075088,2.505021,0
fixedAssetTurnover,4.018195,6.542357,0.529898,2.424742,1.219515,12.273581,3.557024,0.418285,1.895101,2.454876,...,7.511253,0,1.037801,5.511759,105.06599,1.007051,1.826635,0.506995,1.641921,329.620034
grossProfitMargin,0.424126,0.32129,0.168742,0.650505,0.337049,1,0.551846,0.303178,0.301044,0.731747,...,0.468119,0,0.491179,0.208879,0.872603,0.590316,0.342299,-0.291805,0.24805,0.075693
netProfitMargin,0.042668,0.06676,-2.558615,0.078339,0.084526,0.169499,0.142686,-0.357848,0.035051,0.006072,...,0.249329,0,0.122441,-0.032099,0.486494,0.086688,0.090933,-0.952385,-4.233865,-0.002096
returnOnAssets,0.017095,0.076416,-0.25955,0.024159,0.056111,0.007613,0.078168,-0.049829,0.010972,0.003422,...,0.099829,-3.029963,0.017037,-0.019114,0.267108,0.030543,0.066949,-0.409533,-0.46729,-0.023266
returnOnEquity,0.029357,0.152454,-0.62448,0.030734,0.113107,0.091753,0.148253,-0.203403,0.017422,0.009712,...,0.165142,-7.914946,0.043226,-0.079235,0.345775,0.125652,0.152393,-1.118832,-1.981167,-0.03695
debtEquityRatio,0.099413,0.606383,0.031426,0.107397,0.430479,2.790717,0.386602,2.580184,0.324849,0.940369,...,0.416925,1.365636,0.598335,2.469406,0.004572,1.630142,0.624437,0.892108,0.954617,0.000124
interestCoverage,4.038627,4.733129,-25.534192,6.614108,6.908046,0.560153,10.169545,-8.934272,4.335859,3.197208,...,37.271795,-5.451431,1.731732,1.60777,0,4.141383,11.132979,-5.413756,-229.486957,0


In [239]:
fc_data_2023_1

,MBUU,KRUS,WEC,OPBK,HES,LOVE,GCT,PAYO,CLW,SONY,...,TATT,GRVY,DSYWW,BRID,TACT,HPKEW,BRLT,RSVR,SWKH,UCL
currentRatio,1.403961,3.067163,0.54659,0.103461,1.04893,2.184187,1.908924,1.086633,1.972378,0.620535,...,2.399481,5.138386,1.004862,4.864969,3.006674,1.144193,2.6623,1.224756,3.009381,1.252895
quickRatio,0.667468,3.007207,0.39503,1.908157,0.955963,0.833857,1.256373,1.086633,0.854196,0.462832,...,1.095012,5.110256,0.904333,2.608531,1.704312,1.118952,2.171209,1.124684,2.823529,1.197096
inventoryTurnover,6.058041,90.813967,5.74742,0.000028,23.223684,2.548465,3.84159,148324000,5.478424,5.721179,...,1.780928,161.97662,4.254138,4.467971,2.002703,87.401985,5.24701,8.791048,4.687585,19.977554
fixedAssetTurnover,6.779391,0.891199,0.281589,9.879811,0.586035,3.408108,1.661797,20.595817,2.103413,3.642866,...,2.512009,71.472367,1.342739,3.728659,18.053944,0.406959,7.929337,15.431157,6.943729,18.000841
grossProfitMargin,0.253028,0.153535,0.498999,1.000353,0.340249,0.530778,0.267095,0.821534,0.158825,0.272179,...,0.197445,0.331568,0.576707,0.279598,0.510319,0.429481,0.55582,0.607593,0.909004,0.490383
netProfitMargin,0.075278,0.008014,0.149882,0.176129,0.129147,0.040654,0.133708,0.1123,0.051709,0.087114,...,0.041057,0.181966,0.044422,0.013806,0.065372,0.194248,0.001308,0.020764,0.420736,0.032848
returnOnAssets,0.112874,0.00493,0.030335,0.011113,0.057567,0.06336,0.111119,0.012799,0.064422,0.031374,...,0.031734,0.228335,0.023137,0.020803,0.087145,0.068703,0.002135,0.003367,0.04753,0.049658
returnOnEquity,0.17193,0.009123,0.113394,0.124168,0.153795,0.137059,0.324046,0.140504,0.161059,0.139048,...,0.05152,0.285085,0.475586,0.026819,0.120465,0.139024,0.046601,0.007289,0.056676,0.175107
debtEquityRatio,0.016725,0.726186,1.599195,1.152113,0.999666,0.816791,1.388601,0.038511,0.715568,0.562059,...,0.311602,0.053228,1.148136,0.05783,0.098924,0.741168,5.157118,0.916468,0.307732,0.477481
interestCoverage,48.880486,4.811594,2.870959,2.344426,5.897727,333.478632,91.382258,54.707468,5.694534,20.495089,...,3.352347,989.919753,63.196985,7.994819,18.406452,2.93929,0.874025,1.427043,7.923202,30.729316


In [232]:
fc_data_2023_0

,CTVA,MBC,GMED,ICL,ABT,DD,MRK,HPE,RCL,IR,...,OSK,PARR,PPG,ARC,DXLG,SOHO,CAMT,EBR,VZ,CRH
currentRatio,1.56211,1.93818,4.881837,1.529189,1.637887,2.425436,1.251965,0.865917,0.190618,2.216604,...,1.450285,1.304928,1.470321,1.526933,2.227893,1.516479,5.655988,1.763531,0.691693,1.686308
quickRatio,0.899318,1.22324,2.720141,0.883624,1.163211,1.732408,1.004515,0.655379,0.164238,1.668746,...,0.760357,0.500832,1.049466,1.40301,0.882524,1.266819,4.767383,1.747821,0.653045,1.257765
inventoryTurnover,1.43789,7.407126,0.646329,3.004617,2.735921,3.928738,2.536332,4.101585,37.197581,4.356608,...,3.693939,6.039875,5.051716,21.183942,2.937938,19.677804,1.952645,44.311406,26.683034,5.356793
fixedAssetTurnover,4.018195,6.542357,2.424742,1.219515,3.557024,1.895101,2.454876,4.864752,0.4524,9.665589,...,7.233693,5.693537,4.076408,3.812223,3.340134,0.489796,7.511253,1.037801,1.007051,1.826635
grossProfitMargin,0.424126,0.32129,0.650505,0.337049,0.551846,0.301044,0.731747,0.351433,0.336331,0.365716,...,0.183898,0.100525,0.411104,0.335706,0.499412,0.282075,0.468119,0.491179,0.590316,0.342299
netProfitMargin,0.042668,0.06676,0.078339,0.084526,0.142686,0.035051,0.006072,0.069504,0.122086,0.113247,...,0.06198,0.088514,0.069604,0.029285,0.163277,0.022673,0.249329,0.122441,0.086688,0.090933
returnOnAssets,0.017095,0.076416,0.024159,0.056111,0.078168,0.010972,0.003422,0.035431,0.048305,0.050034,...,0.065504,0.188574,0.058669,0.026554,0.254203,0.010018,0.099829,0.017037,0.030543,0.066949
returnOnEquity,0.029357,0.152454,0.030734,0.113107,0.148253,0.017422,0.009712,0.0956,0.359229,0.079591,...,0.16139,0.545626,0.162155,0.053451,0.649452,0.080061,0.165142,0.043226,0.125652,0.152393
debtEquityRatio,0.099413,0.606383,0.107397,0.430479,0.386602,0.324849,0.940369,0.583278,4.554826,0.278379,...,0.230939,0.988355,0.796987,0.645937,1.323126,6.449795,0.416925,0.598335,1.630142,0.624437
interestCoverage,4.038627,4.733129,6.614108,6.908046,10.169545,4.335859,3.197208,13.391026,2.059914,7.430121,...,12.300292,10.276117,6.919028,8.589712,233.410359,1.033827,37.271795,1.731732,4.141383,11.132979


In [233]:
fc_data_2023_1

,MBUU,KRUS,WEC,OPBK,HES,LOVE,GCT,PAYO,CLW,SONY,...,TATT,GRVY,DSYWW,BRID,TACT,HPKEW,BRLT,RSVR,SWKH,UCL
currentRatio,1.403961,3.067163,0.54659,0.103461,1.04893,2.184187,1.908924,1.086633,1.972378,0.620535,...,2.399481,5.138386,1.004862,4.864969,3.006674,1.144193,2.6623,1.224756,3.009381,1.252895
quickRatio,0.667468,3.007207,0.39503,1.908157,0.955963,0.833857,1.256373,1.086633,0.854196,0.462832,...,1.095012,5.110256,0.904333,2.608531,1.704312,1.118952,2.171209,1.124684,2.823529,1.197096
inventoryTurnover,6.058041,90.813967,5.74742,0.000028,23.223684,2.548465,3.84159,148324000,5.478424,5.721179,...,1.780928,161.97662,4.254138,4.467971,2.002703,87.401985,5.24701,8.791048,4.687585,19.977554
fixedAssetTurnover,6.779391,0.891199,0.281589,9.879811,0.586035,3.408108,1.661797,20.595817,2.103413,3.642866,...,2.512009,71.472367,1.342739,3.728659,18.053944,0.406959,7.929337,15.431157,6.943729,18.000841
grossProfitMargin,0.253028,0.153535,0.498999,1.000353,0.340249,0.530778,0.267095,0.821534,0.158825,0.272179,...,0.197445,0.331568,0.576707,0.279598,0.510319,0.429481,0.55582,0.607593,0.909004,0.490383
netProfitMargin,0.075278,0.008014,0.149882,0.176129,0.129147,0.040654,0.133708,0.1123,0.051709,0.087114,...,0.041057,0.181966,0.044422,0.013806,0.065372,0.194248,0.001308,0.020764,0.420736,0.032848
returnOnAssets,0.112874,0.00493,0.030335,0.011113,0.057567,0.06336,0.111119,0.012799,0.064422,0.031374,...,0.031734,0.228335,0.023137,0.020803,0.087145,0.068703,0.002135,0.003367,0.04753,0.049658
returnOnEquity,0.17193,0.009123,0.113394,0.124168,0.153795,0.137059,0.324046,0.140504,0.161059,0.139048,...,0.05152,0.285085,0.475586,0.026819,0.120465,0.139024,0.046601,0.007289,0.056676,0.175107
debtEquityRatio,0.016725,0.726186,1.599195,1.152113,0.999666,0.816791,1.388601,0.038511,0.715568,0.562059,...,0.311602,0.053228,1.148136,0.05783,0.098924,0.741168,5.157118,0.916468,0.307732,0.477481
interestCoverage,48.880486,4.811594,2.870959,2.344426,5.897727,333.478632,91.382258,54.707468,5.694534,20.495089,...,3.352347,989.919753,63.196985,7.994819,18.406452,2.93929,0.874025,1.427043,7.923202,30.729316


In [242]:
fc_data_2023_2

,ULBI,OCC,CCEC,AERTW,SWVLW,SAIC,RSVRW,SWVL,WLFC,ATLCL,...,TVE,SBBA,UZF,GHG,ENO,RCB,RCC,ENJ,WLYB,SEATW
currentRatio,3.801222,4.510428,1.226032,1.643907,0.742451,1.063055,1.224756,0.742451,2.200604,1.362086,...,0.752309,1.130634,1.553829,1.212555,0.603309,0.401394,0.401394,0.572324,0.604408,0.504331
quickRatio,2.02225,1.450991,1.195841,1.177128,0.742451,0.928064,1.124684,0.742451,1.863653,1.165324,...,0.519187,1.11532,1.332963,1.197303,0.471482,0.380399,0.380399,0.320348,0.57009,0.46124
inventoryTurnover,2.85048,2.098754,33.280749,6.74799,18741277,44.842105,8.791048,18741277,2.591493,0.268941,...,6.637324,42.070752,8.683417,46.301248,16.766125,9.343776,9.343776,4.230122,22.534116,9.485298
fixedAssetTurnover,6.421274,10.108912,0.128474,6.06707,18.488063,30.816,15.431157,18.488063,0.188995,50.796572,...,0.331929,0.374859,1.118877,0.692368,0.52212,447.069712,447.069712,0.27407,5.969924,35.676058
grossProfitMargin,0.24149,0.308895,0.487479,0.257199,0.179894,0.115265,0.607593,0.179894,0.739856,0.910734,...,0.374481,0.754832,0.557604,0.41777,0.338103,0.940354,0.940354,0.43873,0.657141,0.720342
netProfitMargin,0.045366,0.028632,0.13092,0.027232,0.133749,0.038941,0.020764,0.133749,0.107313,0.067176,...,0.04148,0.407761,0.013825,0.165503,0.271275,0.299047,0.299047,0.19447,0.008532,0.104556
returnOnAssets,0.04037,0.047095,0.015033,0.042039,0.139592,0.054122,0.003367,0.139592,0.015918,0.02869,...,0.009738,0.129332,0.005023,0.053153,0.109123,0.027284,0.027284,0.039567,0.005543,0.048085
returnOnEquity,0.057423,0.08359,0.040625,0.118622,0.341346,0.177096,0.007289,0.341346,0.089545,0.178632,...,0.031141,0.214159,0.011673,0.186738,0.283777,0.133207,0.133207,0.161166,0.01649,0.709752
debtEquityRatio,0.211582,0.444936,1.527729,0.721657,0.185658,1.491145,0.916468,0.185658,3.694257,4.622042,...,1.347222,0.540991,0.871163,1.271129,0.839723,3.678536,3.678536,1.790606,0.845437,2.557348
interestCoverage,4.699901,0.861633,1.548122,12.6,93.462417,4.175,1.427043,93.462417,2.470151,2.195424,...,1.596591,4.218307,0.709184,23.883074,0.068693,1.218007,1.218007,2.601311,5.536177,5.948908


In [250]:
fc_data_2023_3

,ABLVW,SOHOO,ORAN,RBCP,PCGU,IAA,HFC,LGNDV,AMK
currentRatio,3.100701,1.516479,0.815993,2.952797,0.830715,1.752,2.215298,14.147837,3.838388
quickRatio,1.987423,1.266819,0.778255,1.054316,0.780467,1.424667,1.162215,12.719581,3.838388
inventoryTurnover,6.426604,19.677804,31.604167,1.472314,18.529885,20.032305,9.181038,0.482331,389824000
fixedAssetTurnover,63.2107,0.489796,1.066573,3.526024,0.294601,1.321975,4.582815,13.888313,24.286121
grossProfitMargin,0.248385,0.282075,0.174833,0.411593,0.340061,0.35004,0.160771,0.911959,0.449789
netProfitMargin,0.065437,0.022673,0.055301,0.113456,0.092353,0.139311,0.049732,0.39717,0.173774
returnOnAssets,0.177575,0.010018,0.021455,0.035541,0.017948,0.086949,0.089729,0.062605,0.075973
returnOnEquity,0.8541,0.080061,0.090791,0.065736,0.090096,0.515424,0.156323,0.074409,0.097144
debtEquityRatio,0.40737,6.449795,1.694921,0.568516,2.305631,4.188084,0.311768,0.008796,0.077376
interestCoverage,15.816069,1.033827,3.513755,2.229259,0.937193,8.176471,11.544089,14.971037,21.247365


In [264]:
fc_data_2023 = pd.concat([fc_data_2023_0,fc_data_2023_1, fc_data_2023_2, fc_data_2023_3], axis=1)

In [265]:
fc_data_2023

,CTVA,MBC,GMED,ICL,ABT,DD,MRK,HPE,RCL,IR,...,SEATW,ABLVW,SOHOO,ORAN,RBCP,PCGU,IAA,HFC,LGNDV,AMK
currentRatio,1.56211,1.93818,4.881837,1.529189,1.637887,2.425436,1.251965,0.865917,0.190618,2.216604,...,0.504331,3.100701,1.516479,0.815993,2.952797,0.830715,1.752,2.215298,14.147837,3.838388
quickRatio,0.899318,1.22324,2.720141,0.883624,1.163211,1.732408,1.004515,0.655379,0.164238,1.668746,...,0.46124,1.987423,1.266819,0.778255,1.054316,0.780467,1.424667,1.162215,12.719581,3.838388
inventoryTurnover,1.43789,7.407126,0.646329,3.004617,2.735921,3.928738,2.536332,4.101585,37.197581,4.356608,...,9.485298,6.426604,19.677804,31.604167,1.472314,18.529885,20.032305,9.181038,0.482331,389824000
fixedAssetTurnover,4.018195,6.542357,2.424742,1.219515,3.557024,1.895101,2.454876,4.864752,0.4524,9.665589,...,35.676058,63.2107,0.489796,1.066573,3.526024,0.294601,1.321975,4.582815,13.888313,24.286121
grossProfitMargin,0.424126,0.32129,0.650505,0.337049,0.551846,0.301044,0.731747,0.351433,0.336331,0.365716,...,0.720342,0.248385,0.282075,0.174833,0.411593,0.340061,0.35004,0.160771,0.911959,0.449789
netProfitMargin,0.042668,0.06676,0.078339,0.084526,0.142686,0.035051,0.006072,0.069504,0.122086,0.113247,...,0.104556,0.065437,0.022673,0.055301,0.113456,0.092353,0.139311,0.049732,0.39717,0.173774
returnOnAssets,0.017095,0.076416,0.024159,0.056111,0.078168,0.010972,0.003422,0.035431,0.048305,0.050034,...,0.048085,0.177575,0.010018,0.021455,0.035541,0.017948,0.086949,0.089729,0.062605,0.075973
returnOnEquity,0.029357,0.152454,0.030734,0.113107,0.148253,0.017422,0.009712,0.0956,0.359229,0.079591,...,0.709752,0.8541,0.080061,0.090791,0.065736,0.090096,0.515424,0.156323,0.074409,0.097144
debtEquityRatio,0.099413,0.606383,0.107397,0.430479,0.386602,0.324849,0.940369,0.583278,4.554826,0.278379,...,2.557348,0.40737,6.449795,1.694921,0.568516,2.305631,4.188084,0.311768,0.008796,0.077376
interestCoverage,4.038627,4.733129,6.614108,6.908046,10.169545,4.335859,3.197208,13.391026,2.059914,7.430121,...,5.948908,15.816069,1.033827,3.513755,2.229259,0.937193,8.176471,11.544089,14.971037,21.247365


In [317]:
fc_data_2023.to_csv('fc_data_2023.csv', index=True)

In [267]:
fc_data_2023.columns

Index(['CTVA', 'MBC', 'GMED', 'ICL', 'ABT', 'DD', 'MRK', 'HPE', 'RCL', 'IR',
       ...
       'SEATW', 'ABLVW', 'SOHOO', 'ORAN', 'RBCP', 'PCGU', 'IAA', 'HFC',
       'LGNDV', 'AMK'],
      dtype='object', length=1048)

In [278]:
import yfinance as yf
from datetime import datetime

In [279]:

def get_prices_yf(tickers):
    start_date = "2024-01-01"  
    end_date = datetime.today().strftime('%Y-%m-%d')  
    
    df = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return df

In [296]:
prices = get_prices_yf(list(fc_data_2023.columns))

[*********************100%***********************]  1048 of 1048 completed

12 Failed downloads:
['EAI', 'AERTW', 'SFB', 'EMP', 'ASTLW', 'SWVLW', 'ABLVW', 'DSYWW']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-01-01 -> 2024-11-07)')
['PCGU', 'HFC', 'LGNDV', 'IAA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [297]:
prices

Ticker,A,AAP,ABBV,ABEV,ABG,ABLVW,ABT,ACA,ACEL,ACI,...,XOM,XYL,YETI,YOU,ZBH,ZGN,ZIP,ZTO,ZTS,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02 00:00:00+00:00,138.268341,60.839668,154.183807,2.76,222.839996,NaN,107.700714,81.837875,10.34,22.289557,...,99.796883,112.744019,50.369999,19.855873,120.893456,11.173580,13.57,19.760117,194.642242,28.891378
2024-01-03 00:00:00+00:00,130.704697,59.906696,154.801224,2.75,208.669998,NaN,107.377174,78.864868,9.89,22.172394,...,100.635368,110.651207,47.139999,19.671301,119.977226,11.054923,13.51,20.103769,191.037918,27.968363
2024-01-04 00:00:00+00:00,130.545242,59.405842,155.765976,2.76,207.000000,NaN,108.808601,78.276245,9.83,22.006422,...,99.757889,111.414940,46.080002,20.555296,119.708321,10.867048,13.66,19.559652,192.137054,28.107309
2024-01-05 00:00:00+00:00,130.106781,60.417377,156.421967,2.78,205.539993,NaN,108.632126,78.186455,9.85,22.494585,...,100.060135,111.236404,45.570000,20.953579,119.489227,10.807719,13.81,19.044170,192.939102,27.720240
2024-01-08 00:00:00+00:00,132.916992,60.800388,155.737000,2.79,208.750000,NaN,110.200821,79.134224,9.96,22.445768,...,98.392944,111.930695,46.520000,21.351860,121.471092,10.827496,14.21,18.757792,194.226349,27.839338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-31 00:00:00+00:00,130.309998,35.689999,203.869995,2.18,227.839996,NaN,113.370003,93.639999,11.04,18.100000,...,116.779999,121.779999,35.209999,36.779999,106.919998,7.600000,9.27,23.110001,178.779999,36.099998
2024-11-01 00:00:00+00:00,136.869995,36.250000,203.550003,2.15,227.979996,NaN,118.599998,93.690002,11.02,18.660000,...,114.949997,119.459999,35.419998,36.840000,107.269997,7.600000,9.22,23.090000,181.949997,37.080002
2024-11-04 00:00:00+00:00,139.649994,37.450001,200.470001,2.21,236.979996,NaN,117.650002,95.660004,11.03,18.450001,...,118.610001,120.839996,36.740002,34.770000,108.410004,7.620000,9.07,22.459999,175.179993,37.150002


In [316]:
prices.to_csv('prices.csv', index=True)

In [298]:
start_prices = prices.iloc[0]  
end_prices = prices.iloc[-1]   

yearly_return = (end_prices - start_prices) / start_prices * 100

returns_df = pd.DataFrame(yearly_return, columns=['2024 Return (%)'])

print("2024 Returns (%):")
print(returns_df)

2024 Returns (%):
        2024 Return (%)
Ticker                 
A             -0.353184
AAP          -36.160072
ABBV          30.493597
ABEV         -19.927535
ABG           16.316647
...                 ...
ZGN          -36.009766
ZIP          -26.823872
ZTO           14.827260
ZTS          -12.470185
ZWS           38.138095

[1048 rows x 1 columns]


In [309]:
returns_df = returns_df.T
returns_df

Ticker,A,AAP,ABBV,ABEV,ABG,ABLVW,ABT,ACA,ACEL,ACI,...,XOM,XYL,YETI,YOU,ZBH,ZGN,ZIP,ZTO,ZTS,ZWS
2024 Return (%),-0.353184,-36.160072,30.493597,-19.927535,16.316647,NaN,8.44868,26.054108,11.411995,-15.431245,...,21.246272,9.744184,-28.270792,93.64548,-8.870172,-36.009766,-26.823872,14.82726,-12.470185,38.138095


In [312]:
final = pd.concat([fc_data_2023,returns_df]).dropna(axis=1)

In [313]:
final

,CTVA,MBC,GMED,ICL,ABT,DD,MRK,HPE,RCL,IR,...,UZF,GHG,ENO,RCB,RCC,ENJ,WLYB,SEATW,SOHOO,ORAN
currentRatio,1.56211,1.93818,4.881837,1.529189,1.637887,2.425436,1.251965,0.865917,0.190618,2.216604,...,1.553829,1.212555,0.603309,0.401394,0.401394,0.572324,0.604408,0.504331,1.516479,0.815993
quickRatio,0.899318,1.22324,2.720141,0.883624,1.163211,1.732408,1.004515,0.655379,0.164238,1.668746,...,1.332963,1.197303,0.471482,0.380399,0.380399,0.320348,0.57009,0.46124,1.266819,0.778255
inventoryTurnover,1.43789,7.407126,0.646329,3.004617,2.735921,3.928738,2.536332,4.101585,37.197581,4.356608,...,8.683417,46.301248,16.766125,9.343776,9.343776,4.230122,22.534116,9.485298,19.677804,31.604167
fixedAssetTurnover,4.018195,6.542357,2.424742,1.219515,3.557024,1.895101,2.454876,4.864752,0.4524,9.665589,...,1.118877,0.692368,0.52212,447.069712,447.069712,0.27407,5.969924,35.676058,0.489796,1.066573
grossProfitMargin,0.424126,0.32129,0.650505,0.337049,0.551846,0.301044,0.731747,0.351433,0.336331,0.365716,...,0.557604,0.41777,0.338103,0.940354,0.940354,0.43873,0.657141,0.720342,0.282075,0.174833
netProfitMargin,0.042668,0.06676,0.078339,0.084526,0.142686,0.035051,0.006072,0.069504,0.122086,0.113247,...,0.013825,0.165503,0.271275,0.299047,0.299047,0.19447,0.008532,0.104556,0.022673,0.055301
returnOnAssets,0.017095,0.076416,0.024159,0.056111,0.078168,0.010972,0.003422,0.035431,0.048305,0.050034,...,0.005023,0.053153,0.109123,0.027284,0.027284,0.039567,0.005543,0.048085,0.010018,0.021455
returnOnEquity,0.029357,0.152454,0.030734,0.113107,0.148253,0.017422,0.009712,0.0956,0.359229,0.079591,...,0.011673,0.186738,0.283777,0.133207,0.133207,0.161166,0.01649,0.709752,0.080061,0.090791
debtEquityRatio,0.099413,0.606383,0.107397,0.430479,0.386602,0.324849,0.940369,0.583278,4.554826,0.278379,...,0.871163,1.271129,0.839723,3.678536,3.678536,1.790606,0.845437,2.557348,6.449795,1.694921
interestCoverage,4.038627,4.733129,6.614108,6.908046,10.169545,4.335859,3.197208,13.391026,2.059914,7.430121,...,0.709184,23.883074,0.068693,1.218007,1.218007,2.601311,5.536177,5.948908,1.033827,3.513755


In [315]:
final.to_csv('final.csv', index=True)